In [1]:
import os
import json

from pprint import pprint

from langchain import LLMChain, PromptTemplate
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, AgentType, initialize_agent
from langchain.llms import vertexai
from langchain.tools.base import BaseTool
from langchain.callbacks.manager import (AsyncCallbackManagerForToolRun, CallbackManagerForToolRun)


from vertexai.preview.language_models import TextGenerationModel
from google.cloud import aiplatform

from pydantic import BaseModel, Field, ConfigDict
from typing import Optional, Type

In [2]:
# trace langchain execution
os.environ["LANGCHAIN_TRACING"] = "false"

# authenticate 
GOOGLE_PROJECT_ID = os.getenv("GOOGLE_PROJECT_ID") # ie, confident-jackle-123456
aiplatform.init(project=GOOGLE_PROJECT_ID, location="us-central1")

# square device id presets. More info at https://developer.squareup.com/docs/devtools/sandbox/testing
DEVICE_ID_CHECKOUT_SUCCESS="9fa747a2-25ff-48ee-b078-04381f7c828f"
DEVICE_ID_CHECKOUT_SUCCESS_TIP="22cd266c-6246-4c06-9983-67f0c26346b0"
DEVICE_ID_CHECKOUT_SUCCESS_GC="4mp4e78c-88ed-4d55-a269-8008dfe14e9"
DEVICE_ID_CHECKOUT_FAILURE_BUYER_CANCEL="841100b9-ee60-4537-9bcf-e30b2ba5e215"

# set what device ID to use
os.environ["DEVICE"] = DEVICE_ID_CHECKOUT_SUCCESS_TIP

## overview
In order to maintain customer privacy and minimize the ability for this agent to leak information, it is not going to have any memory (in the prompt). It is still going to be able to retrieve and relay a user's order, but that will be information retrieved from an api using a tool. 

This saves alot of tokens being fed into the input prompt

In [3]:
from tools import OrderTool, GetDetailedMenuTool, FindItemIdTool, MakeOrderCheckoutTool, GetOrderTool

tools = [GetDetailedMenuTool(), FindItemIdTool(), OrderTool(), MakeOrderCheckoutTool(), GetOrderTool()]

In [4]:
agent = initialize_agent(
    tools,
    vertexai.VertexAI(temperature=0),
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [5]:
agent.run("could I order a small blueberry muffin?")



> Entering new AgentExecutor chain...
 Action:
```
{
  "action": "find_item_id_tool",
  "action_input": {
    "name": "small blueberry muffin"
  }
}
```

Observation: QHIFP5QQD373ZLO5P7FHGTCM
Thought: Action:
```
{
  "action": "order_tool",
  "action_input": {
    "item_id": "QHIFP5QQD373ZLO5P7FHGTCM",
    "quantity": "1"
  }
}
```


Observation: Order for Muffin was placed successfully.
Thought: Action:
```
{
  "action": "get_order_tool",
  "action_input": {}
}
```


Observation: ['Muffin']
Thought: Action:
```
{
  "action": "Final Answer",
  "action_input": "Your order contains the following items: Muffin"
}
```

> Finished chain.


'Your order contains the following items: Muffin'

In [6]:
agent.run("could I order a large coffee?")



> Entering new AgentExecutor chain...
 Action:
```
{
  "action": "find_item_id_tool",
  "action_input": {
    "name": "coffee large"
  }
}
```

Observation: JHHYV2RFNB3K2DJGR7OM2RCZ
Thought: Action:
```
{
  "action": "order_tool",
  "action_input": {
    "item_id": "JHHYV2RFNB3K2DJGR7OM2RCZ",
    "quantity": "1"
  }
}
```


Observation: Order for Coffee was placed successfully.
Thought: Action:
```
{
  "action": "Final Answer",
  "action_input": "Your order for a large coffee has been placed."
}
```

> Finished chain.


'Your order for a large coffee has been placed.'

In [7]:
agent.run("what is my current order?")



> Entering new AgentExecutor chain...
 Action:
```
{
  "action": "get_order_tool",
  "action_input": {}
}
```

Observation: ['Muffin', 'Coffee']
Thought: Action:
```
{
  "action": "Final Answer",
  "action_input": "Your current order is a Muffin and a Coffee."
}
```



> Finished chain.


'Your current order is a Muffin and a Coffee.'

In [8]:
agent.run("could I check out?'")



> Entering new AgentExecutor chain...
 Action:
```
{
  "action": "make_order_checkout",
  "action_input": {}
}
```

Observation: Checkout was successful, have a good day!
Thought: Action:
```
{
  "action": "Final Answer",
  "action_input": "Your order has been checked out. Have a good day!"
}
```

> Finished chain.


'Your order has been checked out. Have a good day!'

## Chirp
TODO: setup google chirp just for a simple demo